<a href="https://colab.research.google.com/github/sugarforever/wtf-langchain/blob/main/04_Prompts/04_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 提示词

## 什么是提示词？

提示词（`Prompt`）是指向模型提供的输入。这个输入通常由多个元素构成。`LangChain` 提供了一系列的类和函数，简化构建和处理提示词的过程。
- 提示词模板（Prompt Template）：对提示词参数化，提高代码的重用性。
- 示例选择器（Example Selector）：动态选择要包含在提示词中的示例

## 提示词模板

提示词模板提供了可重用提示词的机制。用户通过传递一组参数给模板，实例化图一个提示词。一个提示模板可以包含：
1. 对语言模型的指令
2. 一组少样本示例，以帮助语言模型生成更好的回复
3. 向语言模型提出的问题

In [ ]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00


In [2]:
from langchain import PromptTemplate
template = """
你精通多种语言，是专业的翻译官。你负责{src_lang}到{dst_lang}的翻译工作。
"""

prompt = PromptTemplate.from_template(template)
prompt.format(src_lang="英文", dst_lang="中文")

'\n你精通多种语言，是专业的翻译官。你负责英文到中文的翻译工作。\n'

### 创建模板

`PromptTemplate` 类是 `LangChain` 提供的模版基础类，它接收两个参数：
1. `input_variables` - 输入变量
2. `template` - 模版

In [3]:
multiple_input_prompt = PromptTemplate(
    input_variables=["color", "animal"],
    template="A {color} {animal} ."
)
multiple_input_prompt.format(color="black", animal="bear")

'A black bear .'

#### 聊天提示词模板

聊天模型，比如 `OpenAI` 的GPT模型，接受一系列聊天消息作为输入，每条消息都与一个角色相关联。这个消息列表通常以一定格式串联，构成模型的输入，也就是提示词。

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

system_template="You are a professional translator that translates {src_lang} to {dst_lang}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt.format_prompt(
    src_lang="English",
    dst_lang="Chinese",
    user_input="Did you eat in this morning?"
).to_messages()

[SystemMessage(content='You are a professional translator that translates English to Chinese.', additional_kwargs={}),
 HumanMessage(content='Did you eat in this morning?', additional_kwargs={}, example=False)]

#### 样本选择器

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 定义反义词示例数据
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

# 定义示例的格式模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 创建基于长度的示例选择器
example_selector = LengthBasedExampleSelector(
    # 可选的样本数据
    examples=examples,
    # 提示词模版
    example_prompt=example_prompt,
    # 格式化的样本数据的最大长度，通过get_text_length函数来衡量
    max_length=25,  # 修正：增加最大长度以显示更多示例
)

# 创建少样本提示模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

print("=== 短输入示例（会选择更多样本）===")
print(dynamic_prompt.format(adjective="big"))

print("\n=== 长输入示例（会选择较少样本）===")
long_input = "big and enormous and huge and massive and gigantic and extremely large"
print(dynamic_prompt.format(adjective=long_input))

Give the antonym of every input

Input: big
Output:


In [ ]:
# 让我们深入了解示例选择器是如何工作的
print("=== 示例选择器工作原理分析 ===")
print(f"空字符串的长度: {example_selector.get_text_length('')}")
print(f"'big'的长度: {example_selector.get_text_length('big')}")
print(f"长字符串的长度: {example_selector.get_text_length(long_input)}")

print("\n=== 选择器为不同输入选择的示例 ===")
# 短输入
short_examples = example_selector.select_examples({"adjective": "big"})
print(f"短输入'big'选择的示例数量: {len(short_examples)}")
for i, example in enumerate(short_examples, 1):
    print(f"  示例{i}: {example}")

# 长输入
long_examples = example_selector.select_examples({"adjective": long_input})
print(f"\n长输入选择的示例数量: {len(long_examples)}")
for i, example in enumerate(long_examples, 1):
    print(f"  示例{i}: {example}")


### 语义相似度示例选择器

除了基于长度的选择器，LangChain还提供基于语义相似度的选择器，它能根据输入的语义相似度来选择最相关的示例。


In [ ]:
# 注意：语义相似度选择器需要安装额外的依赖包
# 这里我们演示如何使用（实际运行需要安装对应的向量数据库和嵌入模型）

# from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
# from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings

# 创建更丰富的示例数据用于语义搜索
semantic_examples = [
    {"input": "happy", "output": "sad", "context": "emotion"},
    {"input": "joyful", "output": "miserable", "context": "emotion"},
    {"input": "tall", "output": "short", "context": "height"},
    {"input": "high", "output": "low", "context": "height"},
    {"input": "fast", "output": "slow", "context": "speed"},
    {"input": "quick", "output": "sluggish", "context": "speed"},
]

print("=== 语义相似度示例选择器概念演示 ===")
print("语义相似度选择器会根据输入与示例的语义相似度来选择最相关的示例")
print("例如：")
print("- 输入'delighted'（高兴的）会选择情感相关的示例如'happy->sad'")
print("- 输入'rapid'（快速的）会选择速度相关的示例如'fast->slow'")
print("- 输入'elevated'（高的）会选择高度相关的示例如'tall->short'")

# 演示基于关键词的简化版本匹配
def simple_semantic_select(input_word, examples, k=2):
    """简化版语义选择器演示"""
    # 定义语义关键词组
    emotion_words = ["happy", "joyful", "delighted", "cheerful", "glad"]
    height_words = ["tall", "high", "elevated", "towering"]
    speed_words = ["fast", "quick", "rapid", "swift"]
    
    # 根据输入词判断类别
    if input_word.lower() in emotion_words:
        category = "emotion"
    elif input_word.lower() in height_words:
        category = "height"
    elif input_word.lower() in speed_words:
        category = "speed"
    else:
        category = "general"
    
    # 选择相关示例
    selected = [ex for ex in examples if ex.get("context", "general") == category]
    return selected[:k] if selected else examples[:k]

# 测试语义选择
test_words = ["delighted", "elevated", "rapid"]
for word in test_words:
    selected = simple_semantic_select(word, semantic_examples)
    print(f"\n输入'{word}'选择的示例:")
    for example in selected:
        print(f"  {example['input']} -> {example['output']} (类别: {example['context']})")


### 实际应用示例：多语言翻译助手

让我们创建一个更实用的提示词模板系统，结合动态示例选择来构建一个智能翻译助手。


In [ ]:
# 创建翻译示例数据
translation_examples = [
    {"english": "Hello, how are you?", "chinese": "你好，你好吗？", "type": "greeting"},
    {"english": "Good morning!", "chinese": "早上好！", "type": "greeting"},
    {"english": "I love programming.", "chinese": "我喜欢编程。", "type": "hobby"},
    {"english": "Python is a great language.", "chinese": "Python是一门很棒的语言。", "type": "technology"},
    {"english": "Let's have dinner together.", "chinese": "我们一起吃晚饭吧。", "type": "social"},
    {"english": "The weather is nice today.", "chinese": "今天天气不错。", "type": "daily"},
]

# 创建翻译示例的模板
translation_example_prompt = PromptTemplate(
    input_variables=["english", "chinese"],
    template="English: {english}\nChinese: {chinese}"
)

# 基于类型的简单选择器
class TypeBasedExampleSelector:
    def __init__(self, examples, k=3):
        self.examples = examples
        self.k = k
    
    def select_examples(self, input_variables):
        # 简单的关键词匹配逻辑
        text = input_variables.get("english", "").lower()
        
        # 根据关键词判断类型
        if any(word in text for word in ["hello", "hi", "morning", "evening"]):
            example_type = "greeting"
        elif any(word in text for word in ["love", "like", "enjoy"]):
            example_type = "hobby"
        elif any(word in text for word in ["python", "programming", "code", "language"]):
            example_type = "technology"
        elif any(word in text for word in ["dinner", "lunch", "eat", "together"]):
            example_type = "social"
        elif any(word in text for word in ["weather", "today", "sunny", "rain"]):
            example_type = "daily"
        else:
            example_type = "general"
        
        # 选择相关示例
        relevant_examples = [ex for ex in self.examples if ex.get("type") == example_type]
        if not relevant_examples:
            relevant_examples = self.examples
        
        return relevant_examples[:self.k]

# 创建智能翻译助手
translation_selector = TypeBasedExampleSelector(translation_examples, k=2)

translation_prompt = FewShotPromptTemplate(
    example_selector=translation_selector,
    example_prompt=translation_example_prompt,
    prefix="""You are a professional English-Chinese translator. 
Based on the following examples, translate the given English text to Chinese.
Pay attention to the context and style of the examples.""",
    suffix="English: {english}\nChinese:",
    input_variables=["english"]
)

# 测试不同类型的翻译
test_sentences = [
    "Good evening, everyone!",
    "I enjoy learning new programming languages.",
    "Python makes coding more efficient.",
    "Would you like to have coffee with me?"
]

print("=== 智能翻译助手演示 ===")
for sentence in test_sentences:
    print(f"\n输入: {sentence}")
    print("生成的提示词:")
    print(translation_prompt.format(english=sentence))
    print("-" * 50)


## 进阶技巧与最佳实践

### 1. 提示词模板的最佳实践

- **模块化设计**：将复杂的提示词分解为多个可重用的组件
- **参数验证**：确保所有必需的参数都被正确传递
- **上下文管理**：根据不同的使用场景选择合适的示例和格式
- **性能优化**：合理设置示例选择器的参数以平衡效果和效率

### 2. 示例选择器的选择策略

- **LengthBasedExampleSelector**：适用于需要控制提示词长度的场景
- **SemanticSimilarityExampleSelector**：适用于需要根据语义相似度选择示例的场景
- **MaxMarginalRelevanceExampleSelector**：平衡相似度和多样性
- **自定义选择器**：根据特定业务逻辑实现定制化的选择策略


In [ ]:
# 演示高级提示词模板技巧

# 1. 带条件逻辑的提示词模板
from langchain import PromptTemplate

conditional_template = """
你是一个{role}。
{%- if difficulty == "beginner" %}
请用简单易懂的语言回答问题。
{%- elif difficulty == "advanced" %}
请提供深入的技术分析和详细解释。
{%- else %}
请提供平衡的回答，既有概述也有一些技术细节。
{%- endif %}

用户问题：{question}
"""

# 注意：上面的模板使用了Jinja2语法，在实际使用中需要相应的模板引擎支持

# 2. 多层次验证的提示词模板
class ValidatedPromptTemplate:
    def __init__(self, template, required_vars, optional_vars=None):
        self.template = template
        self.required_vars = required_vars
        self.optional_vars = optional_vars or []
    
    def format(self, **kwargs):
        # 验证必需参数
        missing_vars = [var for var in self.required_vars if var not in kwargs]
        if missing_vars:
            raise ValueError(f"缺少必需参数: {missing_vars}")
        
        # 设置可选参数的默认值
        for var in self.optional_vars:
            if var not in kwargs:
                kwargs[var] = ""
        
        return self.template.format(**kwargs)

# 创建验证模板
validated_template = ValidatedPromptTemplate(
    template="作为{role}，请回答关于{topic}的问题：{question}。{additional_context}",
    required_vars=["role", "topic", "question"],
    optional_vars=["additional_context"]
)

# 测试验证功能
try:
    # 正确使用
    prompt1 = validated_template.format(
        role="Python专家",
        topic="数据结构",
        question="什么是字典？",
        additional_context="请举例说明。"
    )
    print("✅ 正确的提示词生成:")
    print(prompt1)
    
    # 缺少必需参数的情况
    prompt2 = validated_template.format(
        role="Python专家",
        question="什么是字典？"
    )
except ValueError as e:
    print(f"\n❌ 参数验证失败: {e}")

print("\n" + "="*60)

# 3. 动态角色切换的提示词系统
class RoleBasedPromptSystem:
    def __init__(self):
        self.roles = {
            "teacher": {
                "prefix": "作为一名经验丰富的教师，",
                "style": "教育性的、循序渐进的",
                "examples": ["让我来解释一下...", "首先我们需要理解..."]
            },
            "consultant": {
                "prefix": "作为专业顾问，",
                "style": "分析性的、解决方案导向的",
                "examples": ["基于分析，我建议...", "从商业角度来看..."]
            },
            "friend": {
                "prefix": "作为你的朋友，",
                "style": "轻松友好的、支持性的",
                "examples": ["我觉得...", "你可以试试..."]
            }
        }
    
    def create_prompt(self, role, topic, question):
        role_info = self.roles.get(role, self.roles["teacher"])
        
        prompt = f"""
{role_info['prefix']}我将以{role_info['style']}方式来回答你的问题。

主题：{topic}
问题：{question}

回答方式示例：{role_info['examples'][0]}
"""
        return prompt.strip()

# 测试角色切换系统
role_system = RoleBasedPromptSystem()

test_question = "如何学习机器学习？"
roles_to_test = ["teacher", "consultant", "friend"]

print("=== 动态角色切换演示 ===")
for role in roles_to_test:
    print(f"\n【{role.upper()}角色】")
    prompt = role_system.create_prompt(role, "机器学习", test_question)
    print(prompt)
    print("-" * 40)


## 性能优化与调试技巧

### 1. 提示词长度优化
- 监控生成的提示词长度，避免超出模型的上下文限制
- 使用示例选择器动态调整示例数量
- 合理设置`max_length`参数以平衡效果和效率

### 2. 调试和测试
- 打印生成的完整提示词以验证格式
- 测试边界情况（空输入、超长输入等）
- 验证示例选择的准确性和相关性

### 3. 缓存和重用
- 对于相同参数组合，缓存生成的提示词
- 重用经过验证的示例数据集
- 模块化设计以提高代码复用性


In [ ]:
# 性能优化和调试工具演示

class PromptAnalyzer:
    """提示词分析和优化工具"""
    
    def __init__(self):
        self.metrics = {}
    
    def analyze_prompt(self, prompt_text, name="unnamed"):
        """分析提示词的各项指标"""
        lines = prompt_text.split('\n')
        words = prompt_text.split()
        
        analysis = {
            "name": name,
            "total_length": len(prompt_text),
            "word_count": len(words),
            "line_count": len(lines),
            "avg_line_length": sum(len(line) for line in lines) / len(lines) if lines else 0,
            "max_line_length": max(len(line) for line in lines) if lines else 0
        }
        
        self.metrics[name] = analysis
        return analysis
    
    def compare_prompts(self, prompt1, prompt2, name1="Prompt 1", name2="Prompt 2"):
        """比较两个提示词的指标"""
        analysis1 = self.analyze_prompt(prompt1, name1)
        analysis2 = self.analyze_prompt(prompt2, name2)
        
        print(f"=== 提示词比较: {name1} vs {name2} ===")
        print(f"总长度: {analysis1['total_length']} vs {analysis2['total_length']}")
        print(f"词数: {analysis1['word_count']} vs {analysis2['word_count']}")
        print(f"行数: {analysis1['line_count']} vs {analysis2['line_count']}")
        print(f"平均行长: {analysis1['avg_line_length']:.1f} vs {analysis2['avg_line_length']:.1f}")
        
        return analysis1, analysis2
    
    def suggest_optimization(self, prompt_text):
        """提供优化建议"""
        analysis = self.analyze_prompt(prompt_text)
        suggestions = []
        
        if analysis['total_length'] > 2000:
            suggestions.append("⚠️ 提示词过长，建议缩短或使用示例选择器")
        
        if analysis['max_line_length'] > 100:
            suggestions.append("📝 某些行过长，建议分行提高可读性")
        
        if analysis['word_count'] < 10:
            suggestions.append("💡 提示词可能过于简单，建议添加更多上下文")
        
        return suggestions

# 使用分析工具
analyzer = PromptAnalyzer()

# 分析之前创建的翻译提示词
sample_prompt = translation_prompt.format(english="Hello, how are you today?")
suggestions = analyzer.suggest_optimization(sample_prompt)

print("=== 提示词性能分析 ===")
analysis = analyzer.analyze_prompt(sample_prompt, "翻译助手提示词")

for key, value in analysis.items():
    if key != "name":
        print(f"{key}: {value}")

if suggestions:
    print("\n=== 优化建议 ===")
    for suggestion in suggestions:
        print(suggestion)
else:
    print("\n✅ 提示词指标良好，无需优化")

print("\n=== 调试技巧演示 ===")
print("1. 完整提示词预览:")
print("--- 开始 ---")
print(sample_prompt)
print("--- 结束 ---")

print(f"\n2. 字符计数: {len(sample_prompt)} 字符")
print(f"3. Token估算 (粗略): ~{len(sample_prompt.split()) * 1.3:.0f} tokens")


In [12]:
example_selector.get_text_length("")

1